In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# data prepararion
class CIFAR10Dataset():
    def __init__(self):
        
        # image shape
        self.image_shape = (32,32,3)
        self.num_classes = 10
        
    def get_batch(self):
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        X_train, X_test = [self.preprocess(d) for d in [X_train, X_test]]
        y_train, y_test = \
        [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
        return X_train, y_train, X_test, y_test
      
        
    def preprocess(self, data, label_data=False):
        # for labels
        if label_data:
            # convert class vectors to binary class matrices
            data = keras.utils.to_categorical(data, self.num_classes)
            
        # for pixels data
        else:
            data = data.astype('float32')
            # convert the value to 0~1 scale (pixel's max value = 255)
            data = data / 255
            # add data set length to top
            shape = (data.shape[0], ) + self.image_shape
            data = data.reshape(shape)
            
        return data

In [3]:
# LeNet network
def network(input_shape, num_classes):
    model = Sequential()
    
    # extract image features by convolution and max pooling layers
    model.add(Conv2D(32, kernel_size=3, padding='same',
                    input_shape=input_shape, activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # classifiy the class by fully-connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [ ]:
class Trainer():
    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(loss=loss, optimizer=optimizer,metrics=['accuracy'])
        self.verbose = 1
        self.log_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')),'logdir')
        self.model_file_name = 'model_file.hdf5'
        
    def train(self, X_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            # remove previous execution
            shutil.rmtree(self.log_dir)
        else:
            pass
        os.mkdir(self.log_dir)
        
        self._target.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs, 
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir),
                      ModelCheckpoint(os.path.join(self.log_dir,
                                    self.model_file_name),
                                      save_best_only=True)],
             verbose=self.verbose
        )

In [ ]:
dataset = CIFAR10Dataset()

In [ ]:
# make model
model = network(dataset.image_shape, dataset.num_classes)

W0828 23:30:11.424713 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 23:30:11.466601 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 23:30:11.474579 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0828 23:30:11.533423 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0828 23:30:11.543397 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Ana

In [ ]:
# prepare data
X_train, y_train, X_test, y_test = dataset.get_batch()

# train the model
trainer = Trainer(model, loss='categorical_crossentropy',
                 optimizer=RMSprop())

trainer.train(X_train, y_train, batch_size=50, epochs=10, validation_split=0.2)

W0828 23:30:14.465049 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0828 23:30:14.502943 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0828 23:30:18.362897 12416 deprecation.py:323] From C:\Users\gudeg\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples


W0828 23:30:19.786262 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0828 23:30:19.789254 12416 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
40000/40000 [==============================] - 114s 3ms/step - loss: 1.7627 - acc: 0.3588 - val_loss: 1.3826 - val_acc: 0.4907
Epoch 2/10
40000/40000 [==============================] - 105s 3ms/step - loss: 1.3604 - acc: 0.5121 - val_loss: 1.1989 - val_acc: 0.5754
Epoch 3/10
27750/40000 [===================>..........] - ETA: 30s - loss: 1.2144 - acc: 0.5644

In [ ]:
# show result
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [39]:
os.path.dirname(os.path.abspath('__file__'))

'C:\\Users\\gudeg\\Projects'

In [40]:
os.path.join(os.path.dirname(os.path.abspath('__file__')),'logdir')

'C:\\Users\\gudeg\\Projects\\logdir'